In [ ]:
import time
import websocket
import rel
import _thread

from binance.client import Client

import json

import numpy as np
import pandas as pd

In [ ]:
with open('../data/api.txt', 'r') as file:
    api_key = file.readlines()[0].split("\n")[0]

In [ ]:
with open('../data/api.txt', 'r') as file:
    api_secret = file.readlines()[1].split("\n")[0]

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
symbol = "ETHUSDT" #in uppercase
frequency = "1m" #in lowercase except month
endpoint = f"wss://stream.binance.com:9443/ws/{symbol}@kline_{frequency}"

In [ ]:
#parameters for obtaining historical data

klines_dict = {
    
    "1m" : Client.KLINE_INTERVAL_1MINUTE,
    "3m" : Client.KLINE_INTERVAL_3MINUTE,
    "5m" : Client.KLINE_INTERVAL_5MINUTE,
    "15m" : Client.KLINE_INTERVAL_15MINUTE,
    "30m" : Client.KLINE_INTERVAL_30MINUTE,
    "1h" : Client.KLINE_INTERVAL_1HOUR,
    "2h" : Client.KLINE_INTERVAL_2HOUR,
    "4h" : Client.KLINE_INTERVAL_4HOUR,
    "6h" : Client.KLINE_INTERVAL_6HOUR,
    "8h" : Client.KLINE_INTERVAL_8HOUR,
    "12h" : Client.KLINE_INTERVAL_12HOUR,
    "1d" : Client.KLINE_INTERVAL_1DAY,
    "3d" : Client.KLINE_INTERVAL_3DAY,
    "1w" : Client.KLINE_INTERVAL_1WEEK,
    "1M" : Client.KLINE_INTERVAL_1MONTH

}

#how far back you want historical data? examples:

# 30 minutes ago UTC
# 1 day ago UTC
# 1 Dec, 2017

In [ ]:
def get_data(symbol, timeframe, lookback): #lookback in number of minutes
    
    data = pd.DataFrame(client.get_historical_klines(symbol, timeframe, lookback + " min ago UTC"))
    data.drop([6,9,10,11], inplace=True, axis=1)
    data.columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Volume_quoted", "Num_trades"]
    data = data.set_index("Date")
    data.index = pd.to_datetime(data.index, unit="ms")
    data = data.astype("float")
    
    return data

# returns dataframe all sorted

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.dates as mdates

In [ ]:
%matplotlib notebook 

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
def animated(freq):
    plot_data = get_data(symbol, "1m", "120")
    plt.cla()
    plt.plot(plot_data.index, plot_data.Close)
    plt.title(symbol)
    plt.gcf().autofmt_xdate()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    #plt.tight_layout()

In [ ]:
animation = FuncAnimation(plt.gcf(), animated, 1000)

#plt.tight_layout()
plt.show()

In [ ]:
#video = animation.to_html5_video()
#html = display.HTML(video)
#display.display(html)plt.close()

# to save as an animated video